In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [9]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
train_examples = mnist.train.num_examples
validation_examples = mnist.validation.num_examples
test_examples = mnist.test.num_examples
print(train_examples,validation_examples,test_examples)

batch_size =100
xs,ys = mnist.train.next_batch(batch_size)
print('X shape:',xs.shape)
print('Y shape:',ys.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 5000 10000
X shape: (100, 784)
Y shape: (100, 10)


In [15]:
#tensorflow训练神经网络
import tensorflow as tf

#mnist 数据集的相关常数
INPUT_NODE = 784
OUTPUT_NODE = 10
# 配置神经网络的参数
LAYER1_NODE = 500
BATCH_SIZE =100
LEARNING_RATE_BASE =0.8
LEARNING_RATE_DECAY =0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS =300
MOVING_AVERAGE_DECAY = 0.99

# 一个辅助函数，给定神经网络的输入和所有参数，计算神经网络的前向传播结果
def inference(input_tensor,avg_class,weight1,biases1,weight2,biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weight1)+biases1)
        return tf.matmul(layer1,weight2)+biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weight1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weight2))+avg_class.average(biases2)
    
# 训练模型的过程
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name ='y-input')
    
    #生成隐藏层的参数
    weihgt1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    #生成输出层的参数
    weight2 =tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 =tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #计算在当前参数下神经网络前向传播的结果，这里给出的计算滑动平均类为None,所以含少数不会使用参数的滑动平均值
    y=inference(x,None,weihgt1,biases1,weight2,biases2)
    
#     定义存储训练轮数的变量，这个变量不需要计算滑动平均值，，一般讲代表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0,trainable=False)
    
#     给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类，给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages= tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
#     在所有代表神经网络参数的变量上使用滑动平均，其他辅助变量就不需要了
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    ###########
    #计算使用了滑动平均之后的前向传播结果，滑动平均不会改变变量本身的值，而是会维护一个影子变量来记录其滑动平均值
    average_y = inference(x,variable_averages,weihgt1,biases1,weight2,biases2)
    #计算交叉熵作为刻画预测值与真实值之间差距的损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2 正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    
#     -计算模型的正则化损失，一般只使用权重的正则化损失，而不使用偏置项
    regularization = regularizer(weihgt1)+regularizer(weight2)
#     总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
#     设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples,LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

    #     在训练神经网络时，每更新一遍数据，就会通过反向传播更新擦书，又要更新每一个参数的滑动平均值
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op =tf.no_op(name ='train')

    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    #初始化会话，开始训练
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}

        test_feed = {x:mnist.test.images,y_:mnist.test.labels}

        for i in range(TRAINING_STEPS):
            if i%1000 == 0:
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print('After %d training steps,validation accuracy using average model is %g'%(i,validate_acc))

            xs,ys = mnist.train.next_batch(BATCH_SIZE)  
            sess.run(train_op,feed_dict={x:xs,y_:ys})
            
    #     在训练之后，在测试集上检验神经网络模型的最终正确率
        test_acc = sess.run(accuracy,feed_dict=test_feed)
        print('After %d training steps,test accuracy using average model is %g'%(TRAINING_STEPS,test_acc))
    
    
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)
    train(mnist)
    
if __name__ =='__main__':
    tf.app.run()
        
    


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training steps,validation accuracy using average model is 0.1184
After 300 training steps,test accuracy using average model is 0.9587


SystemExit: 

C:\Program Files (x86)\software\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
import tensorflow as tf

# 定义神经网络相关的参数
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER_NODE = 500

def get_weight_variable(shape,regularizer):
    weights = tf.get_variable("weights",shape,initializer = tf.truncated_normal_initializer(stddev=0.1))
    #当给出了正则化生成函数时，将当前变量的正则化损失加入名字为losses的集合中
    if regularizer != None:
        tf.add_to_collection('losses',regularizer(weights))
    return weights

# 定义神经网络向前传播的过程
def inference(input_tensor,regularizer):
    with tf.variable_scope('layer1',reuse=True):
        weights = get_weight_variable([INPUT_NODE,LAYER_NODE],regularizer)
        biases = tf.get_variable('biases',[LAYER_NODE],initializer=tf.constant_initializer(0.0))
    layer1 = tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
    
# 类似的声明第二层神经网络的变量并完成前向传播的过程
    with tf.variable_scope('layer2',reuse=True):
        weights = get_weight_variable([LAYER_NODE,OUTPUT_NODE],regularizer)
        biases = tf.get_variable('biases',[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
    layer2 =tf.nn.relu(tf.matmul(layer1,weights)+biases)
    return layer2


In [21]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RRATE_DECAY = 0.99
REGULARRIZATON_RATE = 0.0001
TRAINING_STEEPS =300
MOVING_AVERAGE_DECAY = 0.99

# 模型保存的路径和文件名
MODEL_SAVE_PATH ='Mnistdata_Model'
MODEL_NAME ='model.ckpt'

def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name ='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARRIZATON_RATE)
    y = inference(x,regularizer)
    
    global_step = tf.Variable(0,trainable=False)
    variable_average = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op = variable_average.apply(tf.trainable_variables())
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RRATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op = tf.no_op(name='train')
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEEPS):
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            _,loss_value,step = sess.run([train_op,loss,global_step],feed_dict={x:xs,y:ys})
            
            if i%10 ==0:
                print('After %d training step(s),loss on training batch is %g'%(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)
    tf.reset_default_graph()          
                
def main(argv=None):
    mnist =input_data.read_data_sets('MNIST_data/')
    train(mnist)
    
if __name__ == '__main__':
    tf.app.run()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


ValueError: Variable layer1/weight/ExponentialMovingAverage/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\tensorflow\python\framework\ops.py", line 1654, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\tensorflow\python\framework\ops.py", line 3290, in create_op
    op_def=op_def)
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)
